In [2]:
import pandas as pd
from sklearn.externals import joblib as jl
from modules import venn
import re
import numpy as np

%matplotlib inline

In [2]:
# regexp = re.compile(r'[01], [01], [01]')
# dom = None
# data = pd.DataFrame(columns=['ahrefs', 'majestic', 'google', 'domain', 'site'])
# i = -1
# # with open('data/CrawlReport_06_12/Report/1_crawl_results.txt',encoding='utf8') as f:
# with open('data/CrawlReport_06_12/Report/stats_intersect_all.txt',encoding='utf8') as f:
#     contents = list(f.readlines())
#     for line in contents:
#         if '*** ====' in line:
#             dom = line[line.find("*** ==== ")+9:line.find(" ==== ***")]
#             continue
#         if regexp.search(line):
#             i += 1
#             row = line.split(',')
#             row.append(dom)
#             data.loc[i] = row
#             continue
            
# # jl.dump(data, 'data/backlinks_3sources.jl')
# jl.dump(data, 'data/live_3sources.jl')

# data.columns = ['ahrefs', 'majestic', 'google', 'domain', 'site']
# for col in data.columns[:3]:
#     data[col] = data[col].astype(int)
    
# for col in data.columns[3:]:
#     data[col] = data[col].apply(lambda x: x.replace('\n', '').strip())
                                
# data.head()

# # jl.dump(data, 'data/backlinks_3sources.jl')
# jl.dump(data, 'data/live_3sources.jl')

In [3]:
data_live = jl.load('data/backlinks_3sources.jl')
data_live = data_live[data_live.domain != ''].reset_index(drop=True)
data_live.head()

,ahrefs,majestic,google,domain,site
0,1,0,1,academicworks.com,10edges.com
1,0,0,0,alamatpon.com,10edges.com
2,1,0,0,allwomenstalk.com,10edges.com
3,0,1,0,americanbookreview.org,10edges.com
4,0,0,1,answers.com,10edges.com


In [4]:
data_all = jl.load('data/live_3sources.jl')
data_all = data_all[data_live.domain != ''].reset_index(drop=True)
data_all.head()

,ahrefs,majestic,google,domain,site
0,1,1,1,academicworks.com,10edges.com
1,1,0,0,alamatpon.com,10edges.com
2,1,0,1,allwomenstalk.com,10edges.com
3,0,1,0,americanbookreview.org,10edges.com
4,0,0,1,answers.com,10edges.com


In [5]:
len(data_all)

86720

In [6]:
len(data_live)

86720

In [13]:
len(data_all[~(data_all[['ahrefs', 'majestic', 'google']] == 0).all(axis=1)])

86720

In [7]:
len(data_live[~(data_live[['ahrefs', 'majestic', 'google']] == 0).all(axis=1)])

47035

In [29]:
data_all_cnts = data_all.groupby(['site'], as_index=False).agg({'ahrefs': sum, 
                                            'majestic': sum, 
                                            'google': sum})
data_all_cnts.head()

,site,ahrefs,majestic,google
0,10edges.com,44,43,61
1,4quotes4me.co.uk,444,446,247
2,aaronneo.com.au,22,9,60
3,aionhill.com,555,300,547
4,antongorlin.com,393,233,481


In [30]:
data_live = data_live[~(data_live[['ahrefs', 'majestic', 'google']] == 0).all(axis=1)]

In [31]:
data_live_cnts = data_live.groupby(['site'], as_index=False).agg({'ahrefs': sum, 
                                            'majestic': sum, 
                                            'google': sum})
data_live_cnts.head()

,site,ahrefs,majestic,google
0,10edges.com,29,30,34
1,4quotes4me.co.uk,104,87,102
2,aaronneo.com.au,18,8,52
3,aionhill.com,222,170,257
4,antongorlin.com,306,201,353


## What % of referring domains is "live" in each of the three sources

In [35]:
live_per_site = data_live_cnts.merge(data_all_cnts, on=['site'])
live_per_site['ahrefs'] = live_per_site['ahrefs_x'] / live_per_site['ahrefs_y']
live_per_site['majestic'] = live_per_site['majestic_x'] / live_per_site['majestic_y']
live_per_site['google'] = live_per_site['google_x'] / live_per_site['google_y']
live_per_site = live_per_site[['site','ahrefs', 'majestic', 'google']]
live_per_site.head()

,site,ahrefs,majestic,google
0,10edges.com,0.659091,0.697674,0.557377
1,4quotes4me.co.uk,0.234234,0.195067,0.412955
2,aaronneo.com.au,0.818182,0.888889,0.866667
3,aionhill.com,0.400000,0.566667,0.469835
4,antongorlin.com,0.778626,0.862661,0.733888


In [36]:
len(live_per_site)

104

### Calculate the numbers for each of 104 domains separately and then take the average

In [37]:
live_per_site.mean()

ahrefs      0.687055
majestic    0.686169
google      0.696646
dtype: float64

### median

In [38]:
live_per_site.median()

ahrefs      0.756458
majestic    0.743421
google      0.736264
dtype: float64

### consider those 104 domains as 1 and run this study across all data at once

In [45]:
data_live[['ahrefs', 'majestic', 'google']].sum() / data_all[['ahrefs', 'majestic', 'google']].sum()

ahrefs      0.608831
majestic    0.600990
google      0.599185
dtype: float64

### If we consider all unique ref.domains found in 3 sources as 100%, then what % from all these ref.domains each individual source has found?

### Method 1: calculate % of unique ref.domains for each of 104 sites separately, and then take the average and median values across all 104 sites

In [49]:
found_doms1 = data_all.groupby(['site'], as_index=False).agg({'ahrefs': np.mean, 
                                            'majestic': np.mean, 
                                            'google': np.mean})
found_doms1.head()

,site,ahrefs,majestic,google
0,10edges.com,0.478261,0.467391,0.663043
1,4quotes4me.co.uk,0.556391,0.558897,0.309524
2,aaronneo.com.au,0.318841,0.130435,0.869565
3,aionhill.com,0.598060,0.323276,0.589440
4,antongorlin.com,0.548117,0.324965,0.670851


### mean

In [50]:
found_doms1.mean()

ahrefs      0.506517
majestic    0.394779
google      0.608244
dtype: float64

### median

In [51]:
found_doms1.median()

ahrefs      0.514431
majestic    0.402381
google      0.626293
dtype: float64

### Live doms

In [52]:
live_doms1 = data_live.groupby(['site'], as_index=False).agg({'ahrefs': np.mean, 
                                            'majestic': np.mean, 
                                            'google': np.mean})
live_doms1.head()

,site,ahrefs,majestic,google
0,10edges.com,0.547170,0.566038,0.641509
1,4quotes4me.co.uk,0.533333,0.446154,0.523077
2,aaronneo.com.au,0.310345,0.137931,0.896552
3,aionhill.com,0.623596,0.477528,0.721910
4,antongorlin.com,0.573034,0.376404,0.661049


In [53]:
#mean
live_doms1.mean()

ahrefs      0.546705
majestic    0.417289
google      0.661572
dtype: float64

In [54]:
#median
live_doms1.median()

ahrefs      0.562789
majestic    0.423992
google      0.708507
dtype: float64

In [88]:
found_doms = data_all.groupby(['domain'], as_index=False).agg({'ahrefs': max, 
                                                            'majestic': max,
                                                            'google': max})
# found_doms = found_doms[~(found_doms[['ahrefs', 'majestic', 'google']] == 0).all(axis=1)]
found_doms.head()

,domain,ahrefs,majestic,google
0,0-21.co.uk,1,0,0
1,0-3-0.com,1,0,0
2,000.nl,1,1,1
3,000a.biz,0,0,1
4,000webhostapp.com,1,0,0


In [89]:
print('All ref domains: {}'.format(len(found_doms)))

All ref domains: 64194


### all ref domains

In [57]:
found_doms[['ahrefs', 'majestic', 'google']].mean()

ahrefs      0.568200
majestic    0.533851
google      0.582687
dtype: float64

In [73]:
live_doms = data_live.groupby(['domain'], as_index=False).agg({'ahrefs': max, 
                                                            'majestic': max,
                                                            'google': max})
# live_doms = live_doms[~(live_doms[['ahrefs', 'majestic', 'google']] == 0).all(axis=1)]

In [74]:
print('All live ref domains: {}'.format(len(live_doms)))

All live ref domains: 34890


### live ref domain

In [75]:
live_doms[['ahrefs', 'majestic', 'google']].mean()

ahrefs      0.652651
majestic    0.604357
google      0.645343
dtype: float64

### all ref domain

In [76]:
all_ref_doms = len(found_doms)
all_ref_doms

64194

In [77]:
live_ref_doms = len(live_doms)
live_ref_doms

34890

In [78]:
#count all ref domains found by at least one of the 3 sources
print ('count ref domains reported by at least 1 of 3 sources: {}'.format(all_ref_doms))

# unique to ahrefs
print ('unique to ahrefs: {}'.format(len(found_doms[(found_doms.ahrefs == 1) &\
                                (found_doms.majestic == 0) &\
                                (found_doms.google == 0)])))

# unique to majestic
print ('unique to majestic: {}'.format(len(found_doms[(found_doms.ahrefs == 0) &\
                                          (found_doms.majestic == 1) &\
                                          (found_doms.google == 0)])))

# unique to google
print ('unique to google: {}'.format(len(found_doms[(found_doms.ahrefs == 0) &\
                                          (found_doms.majestic == 0) &\
                                          (found_doms.google == 1)])))

# ahrefs and majestic
print ('ahrefs & majestic: {}'.format(len(found_doms[(found_doms.ahrefs == 1) &\
                                                       (found_doms.majestic == 1) &\
                                                       (found_doms.google == 0)])))

# ahrefs and google
print ('ahrefs & google: {}'.format(len(found_doms[(found_doms.ahrefs == 1) &\
                                                     (found_doms.majestic == 0) &\
                                                     (found_doms.google == 1)])))

# majestic and google
print ('majestic & google: {}'.format(len(found_doms[(found_doms.ahrefs == 0) &\
                                                       (found_doms.majestic == 1) &\
                                                       (found_doms.google == 1)])))

# ahrefs and majestic and google
print ('ahrefs & majestic & google: {}'.format(len(found_doms[(found_doms.ahrefs == 1) &\
                                                              (found_doms.majestic == 1) &\
                                                              (found_doms.google == 1)])))

count ref domains reported by at least 1 of 3 sources: 64194
unique to ahrefs: 9226
unique to majestic: 9859
unique to google: 13987
ahrefs & majestic: 7704
ahrefs & google: 6711
majestic & google: 3873
ahrefs & majestic & google: 12834


### Live ref domains

In [79]:
#count all ref domains found by at least one of the 3 sources
print ('count ref domains reported by at least 1 of 3 sources: {}'.format(live_ref_doms))

# unique to ahrefs
print ('unique to ahrefs: {}'.format(len(live_doms[(live_doms.ahrefs == 1) &\
                                (live_doms.majestic == 0) &\
                                (live_doms.google == 0)])))

# unique to majestic
print ('unique to majestic: {}'.format(len(live_doms[(live_doms.ahrefs == 0) &\
                                          (live_doms.majestic == 1) &\
                                          (live_doms.google == 0)])))

# unique to google
print ('unique to google: {}'.format(len(live_doms[(live_doms.ahrefs == 0) &\
                                          (live_doms.majestic == 0) &\
                                          (live_doms.google == 1)])))

# ahrefs and majestic
print ('ahrefs & majestic: {}'.format(len(live_doms[(live_doms.ahrefs == 1) &\
                                                       (live_doms.majestic == 1) &\
                                                       (live_doms.google == 0)])))

# ahrefs and google
print ('ahrefs & google: {}'.format(len(live_doms[(live_doms.ahrefs == 1) &\
                                                     (live_doms.majestic == 0) &\
                                                     (live_doms.google == 1)])))

# majestic and google
print ('majestic & google: {}'.format(len(live_doms[(live_doms.ahrefs == 0) &\
                                                       (live_doms.majestic == 1) &\
                                                       (live_doms.google == 1)])))

# ahrefs and majestic and google
print ('ahrefs & majestic & google: {}'.format(len(live_doms[(live_doms.ahrefs == 1) &\
                                                              (live_doms.majestic == 1) &\
                                                              (live_doms.google == 1)])))

count ref domains reported by at least 1 of 3 sources: 34890
unique to ahrefs: 3542
unique to majestic: 3938
unique to google: 6238
ahrefs & majestic: 4894
ahrefs & google: 4024
majestic & google: 1943
ahrefs & majestic & google: 10311


In [80]:
ahrefs = found_doms[found_doms.ahrefs == 1][['domain']]
ahrefs['source'] = 'AHREFS'

majestic = found_doms[found_doms.majestic == 1][['domain']]
majestic['source'] = 'MAJESTIC'

google = found_doms[found_doms.google == 1][['domain']]
google['source'] = 'GOOGLE'

all_refdoms = pd.concat([ahrefs, majestic, google]).drop_duplicates().reset_index(drop=True)
all_refdoms.head()

,domain,source
0,0-21.co.uk,AHREFS
1,0-3-0.com,AHREFS
2,000.nl,AHREFS
3,000webhostapp.com,AHREFS
4,001casino.com,AHREFS


In [81]:
print('total ahrefs: {}'.format(len(ahrefs)))
print('total majestic: {}'.format(len(majestic)))
print('total google: {}'.format(len(google)))

total ahrefs: 36475
total majestic: 34270
total google: 37405


In [82]:
all_refdoms.source.value_counts()

GOOGLE      37405
AHREFS      36475
MAJESTIC    34270
Name: source, dtype: int64

In [83]:
all_refdoms.to_csv('data/all_refdoms.csv')

### live ref domain

In [84]:
ahrefs = live_doms[live_doms.ahrefs == 1][['domain']]
ahrefs['source'] = 'AHREFS'

majestic = live_doms[live_doms.majestic == 1][['domain']]
majestic['source'] = 'MAJESTIC'

google = live_doms[live_doms.google == 1][['domain']]
google['source'] = 'GOOGLE'

live_refdoms = pd.concat([ahrefs, majestic, google]).drop_duplicates().reset_index(drop=True)
live_refdoms.head()

,domain,source
0,000.nl,AHREFS
1,015.by,AHREFS
2,01net.it,AHREFS
3,020.co.uk,AHREFS
4,0345-number.uk,AHREFS


In [85]:
live_refdoms.to_csv('data/live_refdoms.csv')

In [86]:
print('total ahrefs: {}'.format(len(ahrefs)))
print('total majestic: {}'.format(len(majestic)))
print('total google: {}'.format(len(google)))

total ahrefs: 22771
total majestic: 21086
total google: 22516


In [87]:
live_refdoms.source.value_counts()

AHREFS      22771
GOOGLE      22516
MAJESTIC    21086
Name: source, dtype: int64